In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
import warnings
warnings.filterwarnings("ignore")

In [10]:
df = pd.read_csv(r"data\gemstone.csv" )
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [11]:
cat_cols= list(df.select_dtypes(include=["object"]).columns)
num_cols= list(df.select_dtypes(include=["int64","float64"]).drop(columns=["id",'price'], axis= 1).columns)
num_cols

['carat', 'depth', 'table', 'x', 'y', 'z']

In [12]:
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [13]:
num_pipeline= Pipeline(steps= [("Imputer", SimpleImputer(strategy="mean")), 
                               ("Standard Scaling", StandardScaler())])

cat_pipeline= Pipeline(steps=[("Imputing",SimpleImputer(strategy ='most_frequent')),
                              ("Ordinal Encoding", OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
                              ("Standard Scaling", StandardScaler())])

preprocessor= ColumnTransformer([("num pipeline", num_pipeline, num_cols),
                                ("cat pipelien", cat_pipeline, cat_cols)
])

In [14]:
df.columns

Index(['id', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y',
       'z', 'price'],
      dtype='object')

In [15]:
X= df[['carat', 'cut', 'color', 'clarity', 'depth','table', 'x', 'y', 'z']]
Y= df[['price']]

In [16]:
x_train, x_test, y_train, y_test= train_test_split(X, Y , test_size=25, random_state=34)

In [17]:
X_train_preprocessed =preprocessor.fit_transform(x_train)
X_test_preprocessed =preprocessor.transform(x_test)

# X_train_preprocessed.head()

In [18]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [19]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train_preprocessed,y_train)

    #Make Predictions
    y_pred=model.predict(X_test_preprocessed)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print("\n")

LinearRegression
Model Training Performance
RMSE: 1086.0328844371963
MAE: 716.3114350060554
R2 score 94.65685857465382


Lasso
Model Training Performance
RMSE: 1083.8600797240824
MAE: 711.8781204065532
R2 score 94.6782170242653


Ridge
Model Training Performance
RMSE: 1085.953434672795
MAE: 716.2294854017179
R2 score 94.65764031120293


Elasticnet
Model Training Performance
RMSE: 1608.472700291976
MAE: 1090.963307942133
R2 score 88.27971677353304


